# BlueSky Project
<span style="font-family:Lucida Sans Unicode; font-size:1.3em;">Natural Language Processing</span>

#### Configurazioni

Installazione di Ollama e prime configurazioni.

In [1]:
import requests
import json
import os
import time
import subprocess
import threading
from tqdm import tqdm
import pandas as pd

In [2]:
# Installazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0% 59.7%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Creazione di un thread demone che eseguirà `ollama serve` in modo da avviare un server locale per il servizio Ollama.

In [3]:
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]), daemon=True)
t.start()

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIE2k3cewo+ADBdmc7zMI21OwnzvV/f0QA6O5TpDngIZ0



2024/11/05 14:41:10 routes.go:1158: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-11-05T14:41:10.720Z level=INFO source=images.go:754 msg="total blobs: 

Definiamo alcune variabili e strutture dati utili per il processamento dei post.

In [4]:
# Numero totale di post da processare
total_post = 40000

# Creiamo un DataFrame per memorizzare gli ID dei post che hanno testo nullo o vuoto
null_text_df = pd.DataFrame(columns=['post_id'])

# Path del file in cui sono salvati i post da elaborare
post_filepath = '/kaggle/input/outside-feed-posts/outside_feed_limited_posts.json'

#### Funzioni utili
In questa sezione del notebook definiamo una serie di funzioni utili per facilitare il processo di generazione delle risposte. 
  
Definiamo innanzitutto la funzione `ask_to_llm` che crea il prompt da sottomettere al modello e invia una richiesta all'API locale per generare una risposta in base al prompt.

In [5]:
def ask_to_llm(model, instruction, post, post_id=None):
         
    # Creazione del prompt che viene passato al modello     
    full_prompt = instruction + "Post: " + post
    #print(full_prompt)
    
    # Invio della richiesta e ottenimento della risposta
    response = requests.post('http://localhost:11434/api/generate', 
                             data=json.dumps({'model': model, 'prompt': full_prompt, 'stream': False}))
    
    return response.json().get('response')

Definiamo ora una funzione che legge i post da processare dal file JSON in cui sono salvati, li passa al modello, ottiene la risposta del modello e salva i risultati su un nuovo file JSON. 

In [6]:
def generate_feed_assignments(post_filepath, model, start, end, instruction, results_dir):
    
    start_time = time.time()
    
    # Crea la directory in cui salvare i risultati se essa non esiste già
    if not os.path.isdir(results_dir):
        os.mkdir(results_dir)
    
    with open(post_filepath, 'r') as file:
        data = json.load(file)                              # Carica il file contenente i post in un dizionario
        post_ids = list(data['post_id'].keys())[start:end]  # Prende solo i post il cui id va da start a end
        
        results = [] 

        for post_id in post_ids:
            record_text = data['text'][post_id]             # Estae il testo del post
            
            # Controllo sui post vuoti: se il post ha testo vuoto lo si ignora e si aggiunge il suo id al DataFrame null_text_df
            if record_text is None or record_text.strip() == "":
                global null_text_df
                null_text_df = pd.concat([null_text_df, pd.DataFrame({'post_id': [post_id]})], ignore_index=True)
                print(f"Skipping post ID {post_id} due to null text.")
                continue  

            # Chiama la funzione ask_to_llm per ottenere un'assegnazione di feed per il post corrente
            feed_assignment = ask_to_llm(model, instruction, record_text, post_id=post_id)
            
            # Memorizza il risultato
            results.append({
                'post_id': post_id,
                'text': record_text,
                'feed_assignment': feed_assignment
            })
            print(f"Post ID {post_id} assigned to feed: {feed_assignment} \n")
    
    # Salva i risultati ottenuti
    output_filepath = os.path.join(results_dir, f'post_feed_assignments_{start}_{end}.json')
    with open(output_filepath, 'w') as f:
        json.dump(results, f, indent=4)

    # Stampa info sul tempo di esecuzione    
    end_time = time.time()
    duration = end_time - start_time
    print(f"Duration: {duration} seconds")

## LLAMA 3.1

Abbiamo deciso di utilizzare il modello `llama 3.1`, con 8B parametri. References: https://ollama.com/library/llama3.1:8b.


In [7]:
model="llama3.1"
os.mkdir('/kaggle/working/llama3.1')

time=2024-11-05T14:41:27.180Z level=INFO source=common.go:49 msg="Dynamic LLM libraries" runners="[cpu cpu_avx cpu_avx2 cuda_v11 cuda_v12 rocm_v60102]"
time=2024-11-05T14:41:27.180Z level=INFO source=gpu.go:221 msg="looking for compatible GPUs"
time=2024-11-05T14:41:27.331Z level=INFO source=types.go:123 msg="inference compute" id=GPU-6d9be6a8-6b44-ca95-f979-9adbff5182e1 library=cuda variant=v12 compute=6.0 driver=12.4 name="Tesla P100-PCIE-16GB" total="15.9 GiB" available="15.6 GiB"


In [8]:
# Creiamo un altro thread demone per avvviare il modello llama3.1, così da poterlo successivamente utilizzare
!ollama pull llama3.1     # scarica il modello llama, con 8B parametri
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3.1"]),daemon=True) # il thread avvierà il comando 'ollama run llama3.1'
t2.start()

[GIN] 2024/11/05 - 14:41:29 | 200 |      51.892µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  

time=2024-11-05T14:41:31.059Z level=INFO source=download.go:175 msg="downloading 8eeb52dfb3bb in 16 291 MB part(s)"


pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏ 2.9 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕                ▏  29 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕                ▏  94 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   4% ▕                ▏ 163 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   4% ▕                ▏ 209 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   6% ▕                ▏ 282 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   8% ▕█               ▏ 357 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   9% ▕█               ▏ 400 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  10% ▕█               ▏ 475 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  12% ▕█               ▏ 553 MB/4.7 GB  512 MB/s      8spulling manifest 
pulling 8eeb

time=2024-11-05T14:41:40.688Z level=INFO source=download.go:175 msg="downloading 948af2743fc7 in 1 1.5 KB part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743

...

pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12

time=2024-11-05T14:41:44.097Z level=INFO source=download.go:175 msg="downloading 56bb8bd477a5 in 1 96 B part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B              

time=2024-11-05T14:41:45.866Z level=INFO source=download.go:175 msg="downloading 1a4c3c319823 in 1 485 B part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB              

### Zero-shot prompting  
Analizziamo il comportamento del modello  `llama 3.1` utilizzando lo zero-shot prompting.  

Iniziamo definendo il prompt da fornire al modello.

In [9]:
instruction_zeroshot = """You are a classifier. Assign this post to exactly one of the following 11 topics:
Academic Sky, Black Sky, Book Sky, Disability, Game Dev, Green Sky, News, Political Science, Science, Ukrainian View, What\'s History.
If the post does not fit any of these topics, respond with "Unknown".
Your response must be only the topic name, without any additional text. \n\n"""


[GIN] 2024/11/05 - 14:42:13 | 200 |      27.249µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/11/05 - 14:42:13 | 200 |   24.130951ms |       127.0.0.1 | POST     "/api/show"


⠙ time=2024-11-05T14:42:14.098Z level=INFO source=sched.go:714 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe gpu=GPU-6d9be6a8-6b44-ca95-f979-9adbff5182e1 parallel=4 available=16790978560 required="6.2 GiB"
⠙ time=2024-11-05T14:42:14.191Z level=INFO source=server.go:105 msg="system memory" total="31.4 GiB" free="30.1 GiB" free_swap="0 B"
time=2024-11-05T14:42:14.191Z level=INFO source=memory.go:326 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="6.2 GiB" memory.required.partial="6.2 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[6.2 GiB]" memory.weights.total="4.7 GiB" memory.weights.repeating="4.3 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-11-05T14:42:1

INFO [main] starting c++ runner | tid="139548093931520" timestamp=1730817734
INFO [main] build info | build=10 commit="3a8c75e" tid="139548093931520" timestamp=1730817734
INFO [main] system info | n_threads=2 n_threads_batch=2 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="139548093931520" timestamp=1730817734 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="39029" tid="139548093931520" timestamp=1730817734


llama_model_loader: loaded meta data with 29 key-value pairs and 292 tensors from /root/.ollama/models/blobs/sha256-8eeb52dfb3bb9aefdf9d1ef24b3bdbcfbe82238798c4b918278320b6fcef18fe (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str

INFO [main] model loaded | tid="139548093931520" timestamp=1730817736
[GIN] 2024/11/05 - 14:42:16 | 200 |   3.02143352s |       127.0.0.1 | POST     "/api/generate"


⠏ time=2024-11-05T14:42:16.955Z level=INFO source=server.go:626 msg="llama runner started in 2.76 seconds"


In [10]:
results_dir = '/kaggle/working/llama3.1/zero_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_zeroshot, results_dir)

start = 0, end = 10
[GIN] 2024/11/05 - 14:42:27 | 200 |  649.773031ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:42:28 | 200 |  474.292231ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: News 

Skipping post ID 2 due to null text.
Post ID 3 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:42:28 | 200 |   256.95966ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/05 - 14:42:29 | 200 |  494.742164ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:42:29 | 200 |  224.248909ms |       127.0.0.1 | POST     "/api/generate"
Post ID 5 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:42:29 | 200 |  493.779109ms |       127.0.0.1 | POST     "/api/generate"
Post ID 6 assigned to feed: Book Sky 

[GIN] 2024/11/05 - 14:42:30 | 200 |  471.553762ms |       127.0.0.1 | POST     "/api/generate"
Post ID 7 assigned to feed: Unkn

Proviamo a migliorare le performance fornendo nel prompt anche la descrizione dei feed.

In [11]:
instruction_zeroshot_2 = """You are a classifier. Assign this post to exactly one of the following 11 topics based on their descriptions:
"Academic Sky": Posts related to academia, academic discussions, academic jobs, higher education, scholarships, scientific research, and university.
"Black Sky": Amplifying the voices of any and all Black users.
"Book Sky": A feed for anyone who likes reading and books.
"Disability": Posts discussing disability, accessibility, disability rights, or issues related to disability.
"Game Dev": Posts about all aspects of game development.
"Green Sky": A big list of climate accounts, filtered loosely for keywords.
"News": Headlines from verified news organisations.
"Political Science": A feed for political science and international relations research and discussion.
"Science": The Science Feed. A curated feed from Bluesky professional scientists and science communicators.
"Ukrainian View": Posts from Ukrainians about Ukraine and their experience during the war.
"What's History": Posts by historians using :cardfilebox: or skystorians.

If the post does not fit any of these topics, respond with "Unknown". Your response must be only the topic name, without any additional text."""

In [12]:
results_dir = '/kaggle/working/llama3.1/zero_shot_description'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
   
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_zeroshot_2, results_dir)

start = 0, end = 10
[GIN] 2024/11/05 - 14:42:54 | 200 |  850.897598ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:42:54 | 200 |  473.044656ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: News 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:42:55 | 200 |  279.725721ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Academic Sky 

[GIN] 2024/11/05 - 14:42:55 | 200 |  497.672067ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:42:55 | 200 |   228.25028ms |       127.0.0.1 | POST     "/api/generate"
Post ID 5 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:42:56 | 200 |   497.62752ms |       127.0.0.1 | POST     "/api/generate"
Post ID 6 assigned to feed: Book Sky 

[GIN] 2024/11/05 - 14:42:56 | 200 |  472.313381ms |       127.0.0.1 | POST     "/api/generate"
Post ID 7 assigned to feed:

### One-shot prompting  
Analizziamo il comportamento del modello utilizzando il one-shot prompting.   

Definiamo quindi il prompt formendo anche un esempio di assegnamento di un post al feed corretto.

In [13]:
instruction_oneshot = """You are a classifier. Assign this post to exactly one of the following 11 topics based on their descriptions:
"Academic Sky": Posts related to academia, academic discussions, academic jobs, higher education, scholarships, scientific research, and university.
"Black Sky": Amplifying the voices of any and all Black users.
"Book Sky": A feed for anyone who likes reading and books.
"Disability": Posts discussing disability, accessibility, disability rights, or issues related to disability.
"Game Dev": Posts about all aspects of game development.
"Green Sky": A big list of climate accounts, filtered loosely for keywords.
"News": Headlines from verified news organisations.
"Political Science": A feed for political science and international relations research and discussion.
"Science": The Science Feed. A curated feed from Bluesky professional scientists and science communicators.
"Ukrainian View": Posts from Ukrainians about Ukraine and their experience during the war.
"What's History": Posts by historians using :cardfilebox: or skystorians.

If the post does not fit any of these topics, respond with "Unknown". Your response must be only the topic name, without any additional text.

For example, the post "Seattle child abuse suspect faked death by jumping off bridge then lived in LA" should be assigned to the topic "News". """

In [14]:
results_dir = '/kaggle/working/llama3.1/one_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_oneshot, results_dir)

start = 0, end = 10
[GIN] 2024/11/05 - 14:43:17 | 200 |   518.21331ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:43:17 | 200 |  456.410695ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Book Sky 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:43:18 | 200 |  254.496761ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Disability 

[GIN] 2024/11/05 - 14:43:18 | 200 |  500.512921ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:43:18 | 200 |  209.617606ms |       127.0.0.1 | POST     "/api/generate"
Post ID 5 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:19 | 200 |  498.255397ms |       127.0.0.1 | POST     "/api/generate"
Post ID 6 assigned to feed: Book Sky 

[GIN] 2024/11/05 - 14:43:19 | 200 |  475.181226ms |       127.0.0.1 | POST     "/api/generate"
Post ID 7 assigned to fee

### Two-shot prompting  
Analizziamo il comportamento del modello utilizzando il two-shot prompting.   

Definiamo quindi il prompt formendo due esempi di assegnamenti corretti di post a feed.

In [15]:
instruction_twoshot = """You are a classifier. Assign this post to exactly one of the following 11 topics based on their descriptions:
"Academic Sky": Posts related to academia, academic discussions, academic jobs, higher education, scholarships, scientific research, and university.
"Black Sky": Amplifying the voices of any and all Black users.
"Book Sky": A feed for anyone who likes reading and books.
"Disability": Posts discussing disability, accessibility, disability rights, or issues related to disability.
"Game Dev": Posts about all aspects of game development.
"Green Sky": A big list of climate accounts, filtered loosely for keywords.
"News": Headlines from verified news organisations.
"Political Science": A feed for political science and international relations research and discussion.
"Science": The Science Feed. A curated feed from Bluesky professional scientists and science communicators.
"Ukrainian View": Posts from Ukrainians about Ukraine and their experience during the war.
"What's History": Posts by historians using :cardfilebox: or skystorians.

If the post does not fit any of these topics, respond with "Unknown". Your response must be only the topic name, without any additional text.

For example, the post "Seattle child abuse suspect faked death by jumping off bridge then lived in LA" should be assigned to the topic "News". 
And the post "To be fair, they didn't go quiet. Instead, they blamed women and black people." should be assigned to the topic "Black Sky" """

In [16]:
results_dir = '/kaggle/working/llama3.1/two_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_twoshot, results_dir)

start = 0, end = 10
[GIN] 2024/11/05 - 14:43:33 | 200 |    514.7697ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:33 | 200 |  460.223278ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Book Sky 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:43:34 | 200 |  280.562456ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Academic Sky 

[GIN] 2024/11/05 - 14:43:34 | 200 |  522.454561ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Academic Sky 

[GIN] 2024/11/05 - 14:43:35 | 200 |  252.164533ms |       127.0.0.1 | POST     "/api/generate"
Post ID 5 assigned to feed: Book Sky 

[GIN] 2024/11/05 - 14:43:35 | 200 |  477.570588ms |       127.0.0.1 | POST     "/api/generate"
Post ID 6 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:35 | 200 |  476.327234ms |       127.0.0.1 | POST     "/api/generate"
Post ID 7 assigned to feed: Unknown 



### Few-shot prompting  

Analizziamo il comportamento del modello quando gli forniamo un esempio per ogni feed.

In [17]:
instruction_fewshot = """You are a classifier. Assign this post to exactly one of the following 11 topics based on their descriptions:
"Academic Sky": Posts related to academia, academic discussions, academic jobs, higher education, scholarships, scientific research, and university.
"Black Sky": Amplifying the voices of any and all Black users.
"Book Sky": A feed for anyone who likes reading and books.
"Disability": Posts discussing disability, accessibility, disability rights, or issues related to disability.
"Game Dev": Posts about all aspects of game development.
"Green Sky": A big list of climate accounts, filtered loosely for keywords.
"News": Headlines from verified news organizations.
"Political Science": A feed for political science and international relations research and discussion.
"Science": The Science Feed. A curated feed from Bluesky professional scientists and science communicators.
"Ukrainian View": Posts from Ukrainians about Ukraine and their experience during the war.
"What's History": Posts by historians using :cardfilebox: or skystorians.

If the post does not fit any of these topics, respond with "Unknown". 

Examples of posts assigned to a particular topic:
- Academic Sky: "A colleague suggested not doing group projects bc students can't align their schedules & they're too busy. I already have them work in small groups for activities & I cant give up more class time for group projects during class. Thoughts?"
- Black Sky: "It's tiring cuz Black people are giving so much politically & always have. Those invested in electoralism continue hold up the democratic party, & still get treated as its mules. As radicals fighting US imperialism, Black Americans are at the forefront on Haiti, Palestine, now Cuba"
- Book Sky: "Very much recommend #thehemaneffect by Brian Brown, changed my Perfektion of the (capitalist) world for good. "
- Disability: "Need every Opposition party to come out loud long and strong on this because attacking disability funding is pushing the disabled out of public life."
- Game Dev: "I tried playing a couple of those games once but I kept getting frustrated that they wanted you to dress the character a specific way to get the points to go on. Like I care about what I wanted fashion not what the game devs wanted"
- Green Sky: "I don't think that's true -- offshore wind in NY has now hit the price of the troubled Vogtle nuclear power plant. Full scale seasonal storage doesn't exist yet. And the Royal Society estimate of supply for the UK (a hard case) is 1 kW per capita"
- News: "NASA is marking the first anniversary of the James Webb Space Telescope\u2019s scientific debut with the release of a new image, demonstrating the telescope\u2019s ability to re-envision the universe.\n\n\ud83c\udf0c See more discoveries from the telescope: https://wapo.st/3XQTIWc"
- Political Science: "Looking for recent studies on the use of palate cleansers as a distraction after pre-tests in social science experiments. Wading through a whole bunch of search results on taste tests & tongues. Hoping someone here might be able to more efficiently point me in the right direction"
- Science: "More shots fired in the seaweed-carbon sequestration debate.\n\n\"Without sound science and sufficient knowledge on impacts to these fragile ecosystems, it distracts from more rational and effective blue-carbon interventions."
- Ukrainian View: "I have a memory from my childhood (~2010) when a russian on the Internet said to me:\n\"You don't know exactly who will win elections in your country, but we know who will in ours for sure. It's simple, it's always been like that"
- What's History: "The sculpture park is the third site created by the Equal Justice Initiative in Montgomery, Ala., which is dedicated to taking an unflinching look at the country\u2019s history of slavery, racism and discriminatory policing.\" \ud83d\uddc3\ufe0f www.latimes.com/world-nation..."

Your response must be only the topic name, without any additional text."""

In [18]:
results_dir = '/kaggle/working/llama3.1/few_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_fewshot, results_dir)

start = 0, end = 10
[GIN] 2024/11/05 - 14:43:52 | 200 |  1.974067073s |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:52 | 200 |  483.297844ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: News 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:43:53 | 200 |  241.094422ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: News 

[GIN] 2024/11/05 - 14:43:53 | 200 |  509.294922ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Political Science 

[GIN] 2024/11/05 - 14:43:53 | 200 |  237.226348ms |       127.0.0.1 | POST     "/api/generate"
Post ID 5 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:54 | 200 |  485.514797ms |       127.0.0.1 | POST     "/api/generate"
Post ID 6 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:43:54 | 200 |  491.372564ms |       127.0.0.1 | POST     "/api/generate"
Post ID 7 assigned to feed: Unknown 

[GIN] 20

## QWEN 2.5

 Valutiamo le prestazioni del modello `qwen 2.5` per confrontarle con quelle ottenute da `llama 3.1`.  

Applichiamo a questo modello la stessa varietà di prompt utilizzata per `llama 3.1`, per confrontare i risultati e valutare le differenze di comportamento.


In [19]:
model = 'qwen2.5'
os.mkdir('/kaggle/working/qwen2.5')

In [20]:
# Creiamo un altro thread demone per avvviare il modello qwen2.5, così da poterlo successivamente utilizzare
!ollama pull qwen2.5     # scarica il modello
t3 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "qwen2.5"]),daemon=True)
t3.start()

[GIN] 2024/11/05 - 14:44:01 | 200 |       32.17µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  

time=2024-11-05T14:44:02.497Z level=INFO source=download.go:175 msg="downloading 2bada8a74506 in 16 292 MB part(s)"


pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏ 216 KB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   1% ▕                ▏  42 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   2% ▕                ▏  79 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   4% ▕                ▏ 164 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   6% ▕                ▏ 261 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   7% ▕█               ▏ 307 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   9% ▕█               ▏ 407 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...  10% ▕█               ▏ 491 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...  12% ▕█               ▏ 543 MB/4.7 GB                  pulling manifest 
pulling 2bada8a74506...  13% ▕██              ▏ 621 MB/4.7 GB  621 MB/s      6spulling manifest 
pulling 2bad

time=2024-11-05T14:44:24.237Z level=INFO source=download.go:175 msg="downloading 66b9ea09bd5b in 1 68 B part(s)"


pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b...   0% ▕                ▏    0 B/  68 B                  pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09b

...

pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         
pulling eb4402837c78... 100% ▕████████████████▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕████████████████▏  11 KB                         
pulling 2f15b3218f05... 100% ▕████████████████▏  487 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         
pulling eb4402837c78... 100% ▕████████████████▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕████████████████▏  11 KB                         
pulling 2f15b3218f05... 100% ▕████████████████▏  487 B                         pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B              

⠙ time=2024-11-05T14:44:45.265Z level=INFO source=sched.go:507 msg="updated VRAM based on existing loaded models" gpu=GPU-6d9be6a8-6b44-ca95-f979-9adbff5182e1 library=cuda total="15.9 GiB" available="9.6 GiB"
time=2024-11-05T14:44:45.266Z level=INFO source=sched.go:714 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-2bada8a7450677000f678be90653b85d364de7db25eb5ea54136ada5f3933730 gpu=GPU-6d9be6a8-6b44-ca95-f979-9adbff5182e1 parallel=4 available=10256252928 required="5.6 GiB"
⠹ time=2024-11-05T14:44:45.362Z level=INFO source=server.go:105 msg="system memory" total="31.4 GiB" free="29.7 GiB" free_swap="0 B"
time=2024-11-05T14:44:45.362Z level=INFO source=memory.go:326 msg="offload to cuda" layers.requested=-1 layers.model=29 layers.offload=29 layers.split="" memory.available="[9.6 GiB]" memory.gpu_overhead="0 B" memory.required.full="5.6 GiB" memory.required.partial="5.6 GiB" memory.required.kv="448.0 MiB" memory.required.allocati

INFO [main] starting c++ runner | tid="139055128850432" timestamp=1730817885
INFO [main] build info | build=10 commit="3a8c75e" tid="139055128850432" timestamp=1730817885
INFO [main] system info | n_threads=2 n_threads_batch=2 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="139055128850432" timestamp=1730817885 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="42273" tid="139055128850432" timestamp=1730817885


llama_model_loader: loaded meta data with 34 key-value pairs and 339 tensors from /root/.ollama/models/blobs/sha256-2bada8a7450677000f678be90653b85d364de7db25eb5ea54136ada5f3933730 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:                         general.size_label str              = 7B
llama_model_loader: - kv   6:                            general.license str              

INFO [main] model loaded | tid="139055128850432" timestamp=1730817887
[GIN] 2024/11/05 - 14:44:47 | 200 |  2.769808567s |       127.0.0.1 | POST     "/api/generate"


### Zero-shot prompting  

In [21]:
results_dir = '/kaggle/working/qwen2.5/zero_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_zeroshot, results_dir)

start = 0, end = 5
[GIN] 2024/11/05 - 14:44:51 | 200 |  708.347302ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:44:51 | 200 |  442.889923ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Political Science 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:44:52 | 200 |  269.458806ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:44:52 | 200 |  444.652621ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Political Science 

Duration: 2.2865214347839355 seconds


Eseguiamo anche lo zero-shot prompting in cui forniamo anche la descrizione dei feed.

In [22]:
results_dir = '/kaggle/working/qwen2.5/zero_shot_description'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_zeroshot_2, results_dir)

start = 0, end = 5
[GIN] 2024/11/05 - 14:45:06 | 200 |   829.67378ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:06 | 200 |   418.23853ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Unknown 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:45:06 | 200 |  272.461307ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:07 | 200 |  416.674384ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Unknown 

Duration: 2.300459146499634 seconds


### One-shot prompting

In [23]:
results_dir = '/kaggle/working/qwen2.5/one_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
   
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_oneshot, results_dir)

start = 0, end = 5
[GIN] 2024/11/05 - 14:45:18 | 200 |  438.951739ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:19 | 200 |  417.866296ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Unknown 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:45:19 | 200 |  272.587122ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:19 | 200 |  418.391337ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Unknown 

Duration: 1.9068968296051025 seconds


### Two-shot prompting


In [24]:
results_dir = '/kaggle/working/qwen2.5/two_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_twoshot, results_dir)

start = 0, end = 5
[GIN] 2024/11/05 - 14:45:28 | 200 |  468.147718ms |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Black Sky 

[GIN] 2024/11/05 - 14:45:29 | 200 |  421.870221ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Unknown 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:45:29 | 200 |  278.669003ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:29 | 200 |  427.994259ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Unknown 

Duration: 1.9569060802459717 seconds


### Few-shot prompting

In [25]:
results_dir = '/kaggle/working/qwen2.5/few_shot'

for start in range(0, total_post, 5000):
    end = start + 5000
    print(f"start = {start}, end = {end}")
    
    # Processiamo i post
    generate_feed_assignments(post_filepath, model, start, end, instruction_fewshot, results_dir)

start = 0, end = 5
[GIN] 2024/11/05 - 14:45:42 | 200 |  1.987506188s |       127.0.0.1 | POST     "/api/generate"
Post ID 0 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:42 | 200 |  391.811807ms |       127.0.0.1 | POST     "/api/generate"
Post ID 1 assigned to feed: Unknown 

Skipping post ID 2 due to null text.
[GIN] 2024/11/05 - 14:45:43 | 200 |  284.509521ms |       127.0.0.1 | POST     "/api/generate"
Post ID 3 assigned to feed: Unknown 

[GIN] 2024/11/05 - 14:45:43 | 200 |  433.228444ms |       127.0.0.1 | POST     "/api/generate"
Post ID 4 assigned to feed: Unknown 

Duration: 3.4711127281188965 seconds


## Confronto tra i due modelli

In questa sezione, confrontiamo le classificazioni assegnate dai modelli `llama3.1` e `qwen2.5` sui post per valutare l'affidabilità degli assegnamenti.
Identifichiamo i post concordi e post discordi per analizzare le differenze di classificazione tra i due modelli.  
I post classificati con lo stesso topic da entrambi i modelli sono considerati "ben etichettati", suggerendo una maggiore affidabilità per questi esempi, data la concordanza tra i modelli.


Definiamo una funzione che restituisce le classificazioni concordi e discordi tra i due modelli

In [26]:
def confronta_classificazioni(file_lama, file_qwen):

    with open(file_lama, 'r') as f:
        lama_data = json.load(f)
    with open(file_qwen, 'r') as f:
        qwen_data = json.load(f)
    
    df_lama = pd.DataFrame(lama_data)
    df_qwen = pd.DataFrame(qwen_data)
    
    df = pd.merge(df_lama, df_qwen, on="post_id", suffixes=('_lama', '_qwen'))
    
    df['Concordanza'] = df['feed_assignment_lama'] == df['feed_assignment_qwen']
    
    post_concordi = df[df['Concordanza'] == True]
    post_discordi = df[df['Concordanza'] == False]
    
    return post_concordi, post_discordi


Funzione che calcola la percentuale di concordanza dei modelli in ogni topic.

In [27]:
def concordanza_per_topic(post_concordi, post_discordi):
    
    post = pd.concat([post_concordi, post_discordi])
    
    # Numero totale di post per ciascun topic
    post_per_topic = post['feed_assignment_lama'].value_counts()

    # Numero di post concordi per ciascun topic
    concordi_per_topic = post_concordi['feed_assignment_lama'].value_counts()
    
    percentuale_concordanza = pd.DataFrame({
        "Topic": concordi_per_topic.index,
        "Post Concordi": concordi_per_topic.values,
        "Totale Post": post_per_topic[concordi_per_topic.index].values
    })
    percentuale_concordanza["Percentuale Concordanza"] = (
        percentuale_concordanza["Post Concordi"] / percentuale_concordanza["Totale Post"] * 100
    )

    percentuale_concordanza = percentuale_concordanza.sort_values(by="Percentuale Concordanza", ascending=False)
    
    return percentuale_concordanza


Funzioni per stampare i risultati in tabelle

In [28]:
!pip install rich

In [29]:
from rich.console import Console
from rich.table import Table

def summary(post_concordi, post_discordi, titolo):
    totale = len(post_concordi) + len(post_discordi)
    
    table = Table(title="Confronto Classificazioni "+titolo)
    table.add_column("Tipologia", style="black", justify="left")
    table.add_column("Conteggio", style="cyan", justify="center")
    
    table.add_row("Post Concordi", str(len(post_concordi)))
    table.add_row("Post Discordi", str(len(post_discordi)))
    table.add_row("Totale Post", str(totale))
    
    console = Console()
    console.print(table)

In [30]:
def summary_per_topic(percentuale_concordanza, titolo):
        
    table = Table(title="Percentuale di Concordanza per Topic "+titolo)
    
    table.add_column("Topic", style="blue", justify="left")
    table.add_column("Post Concordi", style="blue", justify="right")
    table.add_column("Totale Post", style="blue", justify="right")
    table.add_column("Percentuale Concordanza (%)", style="cyan", justify="right")
    
    for _, row in percentuale_concordanza.iterrows():
        table.add_row(
            row["Topic"],
            str(row["Post Concordi"]),
            str(row["Totale Post"]),
            f"{row['Percentuale Concordanza']:.2f}"  # Arrotonda a due decimali
        )
    
    console = Console()
    console.print(table)

### Confronto dei risultati ottenuti con lo zero-shot prompting 

In [31]:
file_lama = '/kaggle/input/post-feed-assignments/feed_assigment_result/llama3.1/llama_zero_shot_normal.json'
file_qwen = '/kaggle/input/post-feed-assignments/feed_assigment_result/qwen2.5/qwen_zero_shot_normal.json'

post_concordi, post_discordi = confronta_classificazioni(file_lama, file_qwen)

# Stampa i risultati
summary(post_concordi,post_discordi,"Zero-shot normal")

  Confronto Classificazioni  
      Zero-shot normal       
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Tipologia     ┃ Conteggio ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Post Concordi │   16277   │
│ Post Discordi │   23217   │
│ Totale Post   │   39494   │
└───────────────┴───────────┘

Osserviamo che il numero di post concordi è inferiore rispetto a quelli discordi. I modelli sono in disaccordo, probabilmente perché il prompt, essendo troppo semplice e privo di una descrizione dettagliata dei feed, non è sufficientemente accurato per generare classificazioni corrette.  

Confrontiamo anche i risultati per lo zero-shot prompting in cui è stato fornita anche la descizione dei feed.

In [32]:
file_lama = '/kaggle/input/post-feed-assignments/feed_assigment_result/llama3.1/llama_zero_shot_description.json'
file_qwen = '/kaggle/input/post-feed-assignments/feed_assigment_result/qwen2.5/qwen_zero_shot_description.json'

post_concordi, post_discordi = confronta_classificazioni(file_lama, file_qwen)

# Stampa i risultati
summary(post_concordi,post_discordi,"Zero-shot description")

  Confronto Classificazioni  
    Zero-shot description    
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Tipologia     ┃ Conteggio ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Post Concordi │   21105   │
│ Post Discordi │   18390   │
│ Totale Post   │   39495   │
└───────────────┴───────────┘

In questo caso, il numero di post su cui i modelli concordano è maggiore rispetto a quelli su cui discordano, anche se la differenza tra i due è piuttosto ridotta.

Analizziamo la percentuale di concordanza su ogni topic per verificare se esistono topici su cui i modelli concordano maggiormente.

In [33]:
percentuale_concordanza = concordanza_per_topic(post_concordi, post_discordi)
summary_per_topic(percentuale_concordanza, "Zero-shot description")

           Percentuale di Concordanza per Topic Zero-shot description            
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic             ┃ Post Concordi ┃ Totale Post ┃ Percentuale Concordanza (%) ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Unknown           │         15059 │       16181 │                       93.07 │
│ Science           │           349 │         464 │                       75.22 │
│ Green Sky         │           149 │         249 │                       59.84 │
│ Black Sky         │           957 │        2229 │                       42.93 │
│ What's History    │            61 │         201 │                       30.35 │
│ Disability        │           906 │        3115 │                       29.09 │
│ Political Science │           741 │        2720 │                       27.24 │
│ News              │           613 │        2256 │                       27.17 │
│ Ukrainian View    │            22 │          89 │                       24.72 │
│ Game Dev          │           186 │         806 │                       23.08 │
│ Book Sky          │          1315 │        6424 │                       20.47 │
│ Academic Sky      │           747 │        3852 │                       19.39 │
└───────────────────┴───────────────┴─────────────┴─────────────────────────────┘

Come ci si aspetta, i modelli tendono a concordare maggiormente nell'assegnare i post alla categoria "Unknown". Al contrario, il feed in cui i modelli mostrano meno accordo è "AcademicSky".

### Confronto dei risultati ottenuti con il one-shot prompting 

In [34]:
file_lama = '/kaggle/input/post-feed-assignments/feed_assigment_result/llama3.1/llama_one_shot_description.json'
file_qwen = '/kaggle/input/post-feed-assignments/feed_assigment_result/qwen2.5/qwen_one_shot_description.json'

post_concordi, post_discordi = confronta_classificazioni(file_lama, file_qwen)

# Stampa i risultati
summary(post_concordi,post_discordi,"One-shot description")

  Confronto Classificazioni  
    One-shot description     
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Tipologia     ┃ Conteggio ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Post Concordi │   17856   │
│ Post Discordi │   21639   │
│ Totale Post   │   39495   │
└───────────────┴───────────┘

Anche in questo caso il numero di post su cui i modelli discordano e maggiore rispetto al numero di post su cu i modelli concordano

In [35]:
percentuale_concordanza = concordanza_per_topic(post_concordi, post_discordi)
summary_per_topic(percentuale_concordanza, "One-shot description")

            Percentuale di Concordanza per Topic One-shot description            
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic             ┃ Post Concordi ┃ Totale Post ┃ Percentuale Concordanza (%) ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Unknown           │         12062 │       13112 │                       91.99 │
│ Green Sky         │           101 │         135 │                       74.81 │
│ Science           │           363 │         546 │                       66.48 │
│ Black Sky         │           847 │        1479 │                       57.27 │
│ Political Science │           798 │        2483 │                       32.14 │
│ Game Dev          │           166 │         576 │                       28.82 │
│ What's History    │            30 │         109 │                       27.52 │
│ Disability        │           928 │        3930 │                       23.61 │
│ Ukrainian View    │            11 │          51 │                       21.57 │
│ Academic Sky      │           826 │        4655 │                       17.74 │
│ Book Sky          │          1424 │        9480 │                       15.02 │
│ News              │           300 │        2119 │                       14.16 │
└───────────────────┴───────────────┴─────────────┴─────────────────────────────┘

I modelli mostrano una maggiore concordanza sui post del feed "Green Sky" (dopo "Unknown"), mentre concordano meno sui post del feed "News". Inoltre, osserviamo che, in molti casi, le percentuali di concordanza nei feed risultano generalmente più basse rispetto a quelle riscontrate in precedenza.

### Confronto dei risultati ottenuti con il two-shot prompting 

In [36]:
file_lama = '/kaggle/input/post-feed-assignments/feed_assigment_result/llama3.1/llama_few_shot_description.json'
file_qwen = '/kaggle/input/post-feed-assignments/feed_assigment_result/qwen2.5/qwen_few_shot_description.json'

post_concordi, post_discordi = confronta_classificazioni(file_lama, file_qwen)

# Stampa i risultati
summary(post_concordi,post_discordi,"Two-shot description")

  Confronto Classificazioni  
    Two-shot description     
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Tipologia     ┃ Conteggio ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Post Concordi │   13295   │
│ Post Discordi │   26200   │
│ Totale Post   │   39495   │
└───────────────┴───────────┘

I modelli mostrano un livello di disaccordo ancora maggiore. Fornire un numero maggiore di esempi tende a far aumentare la discordanza tra di essi.

In [37]:
percentuale_concordanza = concordanza_per_topic(post_concordi, post_discordi)
summary_per_topic(percentuale_concordanza, "Two-shot description")

            Percentuale di Concordanza per Topic Two-shot description            
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic             ┃ Post Concordi ┃ Totale Post ┃ Percentuale Concordanza (%) ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Unknown           │          7821 │        9452 │                       82.74 │
│ Green Sky         │            76 │         100 │                       76.00 │
│ Science           │           193 │         271 │                       71.22 │
│ Game Dev          │           148 │         346 │                       42.77 │
│ Black Sky         │           823 │        2489 │                       33.07 │
│ Ukrainian View    │            10 │          37 │                       27.03 │
│ Book Sky          │          2019 │       10046 │                       20.10 │
│ Disability        │          1025 │        5781 │                       17.73 │
│ Political Science │           208 │        1348 │                       15.43 │
│ What's History    │             7 │          48 │                       14.58 │
│ Academic Sky      │           822 │        7227 │                       11.37 │
│ News              │           143 │        1445 │                        9.90 │
└───────────────────┴───────────────┴─────────────┴─────────────────────────────┘

Anche in questo caso, i modelli mostrano una maggiore concordanza sui post del feed "Green Sky", mentre la concordanza è inferiore sui post del feed "News". Inoltre, si riscontra una percentuale di accordo generalmente più bassa, come ci si aspettava.

### Confronto dei risultati ottenuti con il few-shot prompting 

In [38]:
file_lama = '/kaggle/input/post-feed-assignments/feed_assigment_result/llama3.1/llama_feed_shot_description.json'
file_qwen = '/kaggle/input/post-feed-assignments/feed_assigment_result/qwen2.5/qwen_feed_shot_description.json'

post_concordi, post_discordi = confronta_classificazioni(file_lama, file_qwen)

# Stampa i risultati
summary(post_concordi,post_discordi,"Few-shot description")

  Confronto Classificazioni  
    Few-shot description     
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Tipologia     ┃ Conteggio ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Post Concordi │   24488   │
│ Post Discordi │   15007   │
│ Totale Post   │   39495   │
└───────────────┴───────────┘

In questo caso, si osserva un significativo aumento dei post su cui i modelli concordano. Fornire un esempio per ogni feed sembra aiutare i modelli a comprendere meglio la categoria da assegnare.

In [39]:
percentuale_concordanza = concordanza_per_topic(post_concordi, post_discordi)
summary_per_topic(percentuale_concordanza, "Few-shot description")

            Percentuale di Concordanza per Topic Few-shot description            
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic             ┃ Post Concordi ┃ Totale Post ┃ Percentuale Concordanza (%) ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Unknown           │         18154 │       20382 │                       89.07 │
│ Science           │           288 │         380 │                       75.79 │
│ Green Sky         │           162 │         268 │                       60.45 │
│ Black Sky         │          1485 │        3581 │                       41.47 │
│ Disability        │          1092 │        3013 │                       36.24 │
│ News              │           386 │        1170 │                       32.99 │
│ What's History    │            81 │         265 │                       30.57 │
│ Academic Sky      │           955 │        3167 │                       30.15 │
│ Book Sky          │          1200 │        4389 │                       27.34 │
│ Political Science │           502 │        1954 │                       25.69 │
│ Game Dev          │           156 │         727 │                       21.46 │
│ Ukrainian View    │            27 │         131 │                       20.61 │
└───────────────────┴───────────────┴─────────────┴─────────────────────────────┘

I modelli mostrano una maggiore concordanza sui post assegnati alla categoria "Unknown", anche se la percentuale è inferiore rispetto al caso zero-shot. In generale, i modelli evidenziano una percentuale di concordanza più alta in ogni feed rispetto ai casi precedenti.

In definitiva, i modelli mostrano una maggiore concordanza quando viene impiegato il few-shot prompting.